In [1]:
cd ..

d:\Project\ThesisProd


In [2]:
import os
import time
import numpy as np

from dotenv import load_dotenv
from typing import Any, List, Dict, Callable, Union, Optional, Tuple, Literal

from core.llm.TeacherLLM import TeacherBot

In [3]:
llm = TeacherBot()

Custom LLM initialized with model: gpt-3.5-turbo


In [4]:
context = "In Machine Learning, there occurs a process of analyzing data for building or training models. It is just everywhere; from Amazon product recommendations to self-driven cars, it holds great value throughout. As per the latest research, the global machine-learning market is expected to grow by 43% by 2024. This revolution has enhanced the demand for machine learning professionals to a great extent. AI and machine learning jobs have observed a significant growth rate of 75% in the past four years, and the industry is growing continuously. A career in the Machine learning domain offers job satisfaction, excellent growth, insanely high salary, but it is a complex and challenging process. "

In [5]:
respond = llm.create_question(context = context, num_questions = 2, total_options = 4, correct_options = 2)

In [6]:
print(respond)


    [
        ["What is one of the key benefits of pursuing a career in the Machine Learning domain?", "A. Job satisfaction.", "B. Low salary.", "C. Limited growth opportunities.", "D. No demand in the industry.", "Answer: A"],
        ["How much is the global machine-learning market expected to grow by 2024?", "A. 10%", "B. 25%", "C. 43%", "D. 60%", "Answer: C"]
    ]


In [11]:
import json

In [13]:
data_str = respond
data_list = json.loads(data_str)
final_list = [row[:5] for row in data_list]
print(final_list)

[['What is one of the key benefits of pursuing a career in the Machine Learning domain?', 'A. Job satisfaction.', 'B. Low salary.', 'C. Limited growth opportunities.', 'D. No demand in the industry.'], ['How much is the global machine-learning market expected to grow by 2024?', 'A. 10%', 'B. 25%', 'C. 43%', 'D. 60%']]


In [16]:
final_list[0][0]

'What is one of the key benefits of pursuing a career in the Machine Learning domain?'